In [339]:
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from detect_outlier import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import Metric


In [340]:
df_normalized = read_data()
df_normalized.dropna(inplace=True)


In [341]:
def score(label):
    if label == 0:
        return np.random.randint(300, 580)
    if label == 1:
        return np.random.randint(580, 670)
    if label == 2:
        return np.random.randint(670, 740)
    if label == 3:
        return np.random.randint(740, 800)
    if label == 4:
        return np.random.randint(800, 850)
def labeling(scores):
    label = []
    for score in scores:
        if  score < 580:
            label.append(0)
        elif score >=580 and score < 670:
            label.append(1)
        elif score >= 670 and score < 740:
            label.append(2)
        elif score >= 740 and score < 800:
            label.append(3)
        elif score >= 800:
            label.append(4)
    return np.array(label)

- Thử thêm label là điểm ở giữa
- Lấy đầu ra của giải thuật di truyền để làm input

In [354]:
X = np.array(df_normalized[
    [
        "totalAsset",
        "averageTotalAsset",
        "frequencyMountOfTransaction",
        "borrow_per_balance",
        "deposit_per_asset",
        "borrow_per_deposit",
        "totalValueOfLiquidation",
        "numberOfLiquidation",
        "frequencyOfTransaction",
        "frequencyOfDappTransactions",
        "numberOfInteractedDapps",
        "typesOfInteractedDapps",
        "numberOfReputableDapps",
        "age",
    ]
].values)
main_y = df_normalized["1st_label"].values
sub_y = df_normalized["2nd_label"].values

X_train, X_test, main_y_train, main_y_test, sub_y_train, sub_y_test = train_test_split(
    X, main_y, sub_y, stratify=main_y, test_size=0.2, random_state=42
)
y_array2d = np.stack((main_y_train, sub_y_train), axis=1)
y_test_array2d = np.stack((main_y_test, sub_y_test), axis=1)
# Chuẩn bị dữ liệu

In [343]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(14,)),  
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [345]:
class CustomSparseCategoricalCrossentropy(tf.keras.losses.Loss):
    def __init__(
        self,
        from_logits=False,
        reduction=tf.keras.losses.Reduction.AUTO,
        name="custom_sparse_categorical_crossentropy",
    ):
        super().__init__(reduction=reduction, name=name)
        self.from_logits = from_logits

    def call(self, y_true, y_pred):
        # Calculate the standard sparse categorical crossentropy loss
        first_label = tf.cast(y_true[:, 0], tf.int32)
        sencond_label = tf.cast(y_true[:, 1], tf.int32)
        first_scce_loss = tf.keras.losses.sparse_categorical_crossentropy(
            first_label, y_pred, from_logits=self.from_logits
        )
        sencond_scce_loss = tf.keras.losses.sparse_categorical_crossentropy(
            sencond_label, y_pred, from_logits=self.from_logits
        )

        return first_scce_loss
class CustomAccuracy(Metric):
    def __init__(self, **kwargs):
        super(CustomAccuracy, self).__init__(**kwargs)
        self.correct = self.add_weight("correct", initializer="zeros")
        self.total = self.add_weight("total", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=1)
        # Cast y_true to the same dtype as y_pred
        first_label = tf.cast(y_true[:, 0], y_pred.dtype)
        second_label = tf.cast(y_true[:, 1], y_pred.dtype)
        # Check the equality of the prediction and truth
        values = tf.cast(tf.logical_or(tf.equal(y_pred, first_label), tf.equal(y_pred, second_label)), 'float32')
        self.correct.assign_add(tf.reduce_sum(values))
        self.total.assign_add(tf.cast(tf.size(first_label), 'float32'))

    def result(self):
        return self.correct / self.total

    def reset_state(self):
        self.correct.assign(0)
        self.total.assign(0)

# Example usage of the custom loss function
custom_accuracy = CustomAccuracy()
custom_loss = CustomSparseCategoricalCrossentropy()

In [346]:
# model.compile(optimizer='adam', loss='mse', metrics=['mae'])
# model.fit(X_train, main_y_train, epochs=15, batch_size=32)

model.compile(optimizer='adam', loss=custom_loss, metrics=[custom_accuracy])
model.fit(X_train, y_array2d , epochs=15, batch_size=32)

Epoch 1/15
2859/2859 [==============================] - 3s 940us/step - loss: 2.1311 - custom_accuracy_10: 0.8883
Epoch 2/15
2859/2859 [==============================] - 3s 896us/step - loss: 0.5833 - custom_accuracy_10: 0.9132
Epoch 3/15
2859/2859 [==============================] - 2s 867us/step - loss: 0.3372 - custom_accuracy_10: 0.9212
Epoch 4/15
2859/2859 [==============================] - 3s 898us/step - loss: 0.2496 - custom_accuracy_10: 0.9264
Epoch 5/15
2859/2859 [==============================] - 3s 886us/step - loss: 0.2156 - custom_accuracy_10: 0.9294
Epoch 6/15
2859/2859 [==============================] - 3s 964us/step - loss: 0.1976 - custom_accuracy_10: 0.9335
Epoch 7/15
2859/2859 [==============================] - 3s 1ms/step - loss: 0.1838 - custom_accuracy_10: 0.9374
Epoch 8/15
2859/2859 [==============================] - 3s 883us/step - loss: 0.1785 - custom_accuracy_10: 0.9382
Epoch 9/15
2859/2859 [==============================] - 3s 897us/step - loss: 0.1718 - cus

In [368]:
# X_new = np.expand_dims(X_test, axis=0)
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)
print("Dự đoán:", predicted_labels)

715/715 [==============================] - 1s 840us/step
Dự đoán: [2 1 1 ... 1 1 1]


In [369]:
def new_accuracy(y_train, sub_y_train, y_pred):
    y_train = np.array(y_train)
    sub_y_train = np.array(sub_y_train)
    y_pred = np.array(y_pred)
    main_label_count = np.sum(y_pred == y_train)
    condition = np.logical_or(y_pred == sub_y_train, y_pred == y_train)
    count = np.sum(condition)
    accuracy = count / len(y_train)
    return accuracy, main_label_count, count - main_label_count
print(new_accuracy(y_test_array2d[:, 0], y_test_array2d[:, 1], predicted_labels))

(0.9498863040055974, 21603, 119)


In [367]:
y_test_array2d[:, 0]

array([2., 1., 1., ..., 1., 1., 1.])

In [351]:
import numpy as np


# Sử dụng numpy.unique để lấy các nhãn duy nhất và số lần xuất hiện của chúng
unique_labels, counts = np.unique(predicted_labels, return_counts=True)

# Kết hợp kết quả vào một dictionary hoặc một mảng hai chiều để dễ dàng truy cập
label_count_dict = dict(zip(unique_labels, counts))
print(label_count_dict)

{0: 91, 1: 19181, 2: 3079, 3: 216, 4: 301}
